# Adicionando funções externas a API da OpenAI

Um grande salto de possibilidades de utilizações únicas da LLMs ocorrreu quando a OpenAI lançou o function calling. Essa ferramenta permite adicionarmos manualmente funções externas ao modelo que ele, dependendo da situação, poderá utilizar para obter novas informações ou atuar em diversos escopos. Vamos fazer uma breve revisão de como utilizamos funções externas na api da OpenAI, este assunto é explorado mais afundo no curso de Explorando a API da OpenAI. Na próxima aula, mostraremos como o framework langChain facilita a utilização das funções externas.

## Importações iniciais

In [ ]:
import json
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

## Criando função que será adicionada ao modelo

Utilizaremos uma função simples que simula uma api de tempo, que retorna a temperatura de um determinado local. Lembrando que modelos de llm são treinados com dados históricos, portanto, não possuem informações atuais. A única forma de eles entenderem o que está ocorrendo neste instante é passando informações pra eles através de prompts ou de funções externas.

In [2]:
def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
        )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
        )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
        )

In [3]:
obter_temperatura_atual("Porto Alegre")

'{"local": "Porto Alegre", "temperatura": "25", "unidade": "celsius"}'

## Criando descrição da função

Através dessa descrição o modelo entenderá o que a função faz e como ela pode ser utilizada

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]

## Chamando o modelo com a nova ferramenta

Para chamar o modelo com a ferramenta criada, basta passar o argumento tools com uma lista de ferramentas.

In [40]:
mensagens = [
    {'role': 'user', 'content': 'Qual é temperatura em Porto Alegre agora?'}
]

resposta = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=mensagens,
    tools=tools,
    tool_choice='auto'
)
resposta

ChatCompletion(id='chatcmpl-BhiE2bOZKggRbXeWwEeZbZVg60glJ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SNTy4NdVfTilN11TeUtalrhq', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]))], created=1749757806, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=27, prompt_tokens=82, total_tokens=109, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

### Analisando a resposta

Podemos perceber que o conteúdo da resposta veio vazio, pois para a pergunta "Qual é a temperatura em Porto Alegre?" ele necessitará chamar a função antes.

In [41]:
mensagem = resposta.choices[0].message
mensagem

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SNTy4NdVfTilN11TeUtalrhq', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')])

In [42]:
mensagem.content

In [43]:
mensagem.tool_calls

[ChatCompletionMessageToolCall(id='call_SNTy4NdVfTilN11TeUtalrhq', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]

In [44]:
tool_calls = mensagem.tool_calls[0]
print(tool_calls.function.name)
print(tool_calls.function.arguments)

obter_temperatura_atual
{"local":"Porto Alegre","unidade":"celsius"}


### Adicionando resultado da função as mensagens

In [45]:
observacao = obter_temperatura_atual(**json.loads(tool_calls.function.arguments))
observacao

'{"local": "Porto Alegre", "temperatura": "25", "unidade": "celsius"}'

### Chamando novamente o modelo

In [46]:
mensagens.append(mensagem)
mensagens

[{'role': 'user', 'content': 'Qual é temperatura em Porto Alegre agora?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SNTy4NdVfTilN11TeUtalrhq', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')])]

In [47]:
mensagens.append({
    'tool_call_id': tool_calls.id,
    'role': 'tool',
    'name': tool_calls.function.name,
    'content': observacao
})
mensagens

[{'role': 'user', 'content': 'Qual é temperatura em Porto Alegre agora?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SNTy4NdVfTilN11TeUtalrhq', function=Function(arguments='{"local":"Porto Alegre","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]),
 {'tool_call_id': 'call_SNTy4NdVfTilN11TeUtalrhq',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "Porto Alegre", "temperatura": "25", "unidade": "celsius"}'}]

In [48]:
resposta = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=mensagens,
    tools=tools,
    tool_choice='auto'
)
resposta

ChatCompletion(id='chatcmpl-BhiEHmGOVbrROieZCkbzhaMbN5PMy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A temperatura em Porto Alegre agora é de 25°C.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1749757821, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=153, total_tokens=168, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [49]:
resposta.choices[0].message.content

'A temperatura em Porto Alegre agora é de 25°C.'

## Explorando diferentes perguntas e o parâmetro tool_choice

Através do parâmetro tool_choice é possível forçar o modelo a sempre utilizar uma tool. Vamos ver como ele se comporta para diferentes perguntas modificando o parâmetro.

### Parâmetro "auto"

Assim o modelo define automaticamente se é necessária a utilização de uma função ou não

In [51]:
mensagens = [
    {'role': 'user', 'content': 'Qual é temperatura em Porto Alegre agora?'}
]

resposta = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=mensagens,
    tools=tools,
    tool_choice='auto'
)
mensagem = resposta.choices[0].message
print('Conteudo: ', mensagem.content)
print('Tolls: ', mensagem.tool_calls)

Conteudo:  None
Tolls:  [ChatCompletionMessageToolCall(id='call_wS8kcCtbC9XAJnWArVrU7T37', function=Function(arguments='{"local":"Porto Alegre"}', name='obter_temperatura_atual'), type='function')]


In [52]:
mensagens = [
    {'role': 'user', 'content': 'Olá'}
]

resposta = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=mensagens,
    tools=tools,
    tool_choice='auto'
)
mensagem = resposta.choices[0].message
print('Conteudo: ', mensagem.content)
print('Tolls: ', mensagem.tool_calls)

Conteudo:  Olá! Como posso ajudar você hoje?
Tolls:  None


### Parâmetro "none"

Com o parâmetro "none", o modelo não vai utilizar funções.

In [53]:
mensagens = [
    {'role': 'user', 'content': 'Qual é temperatura em Porto Alegre agora?'}
]

resposta = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=mensagens,
    tools=tools,
    tool_choice='none'
)
mensagem = resposta.choices[0].message
print('Conteudo: ', mensagem.content)
print('Tolls: ', mensagem.tool_calls)

Conteudo:  Por favor, aguarde um momento enquanto verifico a temperatura atual em Porto Alegre.
Tolls:  None


In [54]:
mensagens = [
    {'role': 'user', 'content': 'Olá'}
]

resposta = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=mensagens,
    tools=tools,
    tool_choice='none'
)
mensagem = resposta.choices[0].message
print('Conteudo: ', mensagem.content)
print('Tolls: ', mensagem.tool_calls)

Conteudo:  Olá! Como posso ajudar você hoje?
Tolls:  None


### Parâmetro "function"

Podemos fazer o modelo rodar obrigatoriamente a função, passando dentro de um dicionário a função que o modelo deve rodar.

In [55]:
mensagens = [
    {'role': 'user', 'content': 'Qual é temperatura em Porto Alegre agora?'}
]

resposta = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=mensagens,
    tools=tools,
    tool_choice={'type': 'function', 'function':{'name': 'obter_temperatura_atual'}}
)
mensagem = resposta.choices[0].message
print('Conteudo: ', mensagem.content)
print('Tolls: ', mensagem.tool_calls)

Conteudo:  None
Tolls:  [ChatCompletionMessageToolCall(id='call_69zf1w5E8N4RVcpB7LjX5QDF', function=Function(arguments='{"local":"Porto Alegre"}', name='obter_temperatura_atual'), type='function')]


In [56]:
mensagens = [
    {'role': 'user', 'content': 'Olá'}
]

resposta = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=mensagens,
    tools=tools,
    tool_choice={'type': 'function', 'function':{'name': 'obter_temperatura_atual'}}
)
mensagem = resposta.choices[0].message
print('Conteudo: ', mensagem.content)
print('Tolls: ', mensagem.tool_calls)

Conteudo:  None
Tolls:  [ChatCompletionMessageToolCall(id='call_WNMZIUN9xZEYWrepi7FQHU2J', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]
